In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read data

In [ ]:
input_path = '/kaggle/input/tabular-playground-series-feb-2021'
train = pd.read_csv(os.path.join(input_path,'train.csv'), index_col='id')
display(train.tail())

In [ ]:
test = pd.read_csv(os.path.join(input_path,'test.csv'), index_col='id')
display(test.tail())

In [ ]:
train.info()

In [ ]:
test.info()

## Categorical Data

In [ ]:
for i in range(10):
    print(train[f'cat{i}'].value_counts())

In [ ]:
for i in range(10):
    print(test[f'cat{i}'].value_counts())

In [ ]:
train.describe()

In [ ]:
test.describe()

# Taking care of missing data

In [ ]:
print(train.isnull().sum())

In [ ]:
print(test.isnull().sum())

Missing data is nothing.

# Split X and y

In [ ]:
X = train.iloc[:, :-1].values
y = train.iloc[:, -1].values

In [ ]:
print(len(X),type(X), len(y),type(y), len(test.values),type(test.values))

# Encoding categorical data

## One hot encoding

Concatenate for one hot

In [ ]:
X_and_test =np.concatenate([X, test.values])

In [ ]:
print(len(X_and_test))

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0,1,2,3,4,5,6,7,8,9])], remainder='passthrough')
X_and_test = ct.fit_transform(X_and_test)

In [ ]:
X_and_test.shape

In [ ]:
X_split = np.split(X_and_test, [len(X)])
X = X_split[0]
test = X_split[1]

In [ ]:
print(X.shape,test.shape)

## Seapalate to feature and target

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.60)

# Training

## LightGBM

In [ ]:
from optuna.integration import lightgbm as lgb

dtrain = lgb.Dataset(X_train, label=y_train)
eval_data = lgb.Dataset(X_test, label=y_test)

param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
    }

best = lgb.train(param, 
                 dtrain,
                 valid_sets=eval_data,
                 early_stopping_rounds=100)

In [ ]:
print(best.params)
print(best.best_iteration)
print(best.best_score)

# Make submission

In [ ]:
submission = pd.read_csv(os.path.join(input_path,'sample_submission.csv'), index_col='id')

In [ ]:
import lightgbm as lgbm
lgb_params = best.params
lgb_params.pop('early_stopping_round') # エラーになるので削除
lgb_train = lgbm.Dataset(
                 X, 
                 y)

lgb = lgbm.train(lgb_params, lgb_train)
test_pred = lgb.predict(test)

In [ ]:
submission['target'] = test_pred
print(submission)
submission.to_csv('lightgbm.csv')